<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/Kaggle/Tabular%20Playground%20Feb/tab_playground_feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>